## Calls For Service
#### Questions to ask:
<b>Descriptive Analysis</b>
- What are the most common call types this year vs. last year?
- What times of day see the highest volume of calls by shift?
- Which addresses have the highest number of repeated calls?
- What percentage of calls resulted in a report being taken or an arrest?

<b>Trend & Comparative Analysis</b>
- How do weekly call volumes compare to the same week last year and to the 5-year average?
- Are we seeing a year-over-year increase in disturbance calls?
- What are the call volume trends for mental health or overdose-related incidents?
- How have reportable vs. non-reportable calls trended over time?
- Is there an increase in traffic-related calls during certain weather events?

<b>Spatial Analysis</b>
- What are the top 10 hotspots by call volume in the past 7, 30, and 90 days?
- Which areas have the highest concentration of violent crime calls?
- Are there emerging hotspots not present in previous periods?

<b>Operational Efficiency</b>
- What is the average time between call creation and unit arrival for high-priority calls?
- How many calls per shift are handled without report, arrest, or further follow-up?
- Which call types consume the most officer time with least actionable outcome?
- Are officers or shifts disproportionately handling more serious incidents?

In [1]:
import os
import pandas as pd
import inc.functions as fn
from inc.credential_manager import inject_decrypted_env, get_passphrase  # << New import
import h3

# Obtain the passphrase from hidden file or user input
passphrase = get_passphrase()

# Inject decrypted environment variables
inject_decrypted_env(environment="prod", passphrase=passphrase)

True

### Production View

In [2]:
# cfs_df = fn.fetch_calls_for_service(ori="OH0760400",start_date=fn.yesterday("str"), end_date=fn.today("str"), data_type="All")

In [3]:
# fn.get_cliff_notes(cfs_df.loc[cfs_df['IncidentNumber'] == "2025-00022512"]['notes'].iloc[-1])

In [4]:
# for col in cfs_df.select_dtypes(include=['datetimetz']).columns:
#     cfs_df[col] = cfs_df[col].dt.tz_localize(None)
# cfs_df.to_excel("resources/cfs/CFS_five_years.xlsx", index=False, header=True)

### Development View

In [7]:
cfs_df = pd.read_excel("resources/cfs/cfs_past_five_years.xlsx")

# Already done in fetch_calls_for_service
# Process the DataFrame
# cfs_df = fn.preprocess_calls(cfs_df)
fn.update_daily_summary(cfs_df, csv_filename="resources/cfs/call_type_daily_summary.csv")

No new data to process.


In [8]:
cfs_df["Hex_ID_10"] = cfs_df.apply(
    lambda row: fn.safe_latlng_to_hex(row["LatitudeY"], row["LongitudeX"], resolution=10),
    axis=1
)
cfs_df["Hex_ID_7"] = cfs_df.apply(
    lambda row: fn.safe_latlng_to_hex(row["LatitudeY"], row["LongitudeX"], resolution=7),
    axis=1
)

## Questions

What are the most common call types this year vs. last year?

In [17]:
# df = cfs_df.loc[(cfs_df['Reportable'] == True) & (cfs_df['ORI'] == "OH0760400")].copy()
# def most_common_call_types_this_vs_last_year(df, top_n=10):
#     df = df.copy()
#     df['LocalDatetime'] = pd.to_datetime(df['LocalDatetime'], errors='coerce')
#     df['Year'] = df['LocalDatetime'].dt.year

#     current_year = pd.Timestamp.today().year
#     last_year = current_year - 1

#     this_year_calls = (
#         df[df['Year'] == current_year]['CallType']
#         .value_counts()
#         .head(top_n)
#         .rename('ThisYear')
#     )

#     last_year_calls = (
#         df[df['Year'] == last_year]['CallType']
#         .value_counts()
#         .head(top_n)
#         .rename('LastYear')
#     )

#     # Combine both for comparison
#     comparison_df = pd.concat([this_year_calls, last_year_calls], axis=1).fillna(0).astype(int)
#     comparison_df['Change'] = comparison_df['ThisYear'] - comparison_df['LastYear']
#     comparison_df['% Change'] = ((comparison_df['Change'] / comparison_df['LastYear'].replace(0, 1)) * 100).round(1)

#     return comparison_df.sort_values(by='ThisYear', ascending=False)

# result = most_common_call_types_this_vs_last_year(df)
# print(result)

In [9]:
import numpy as np
df = cfs_df.loc[(cfs_df['Reportable'] == True) & (cfs_df['ORI'] == "OH0760400")].copy()
def most_common_call_types_asof(
    df,
    as_of=None,
    top_n=10,
    datetime_col="LocalDatetime",
    calltype_col="CallType",
    tz="America/New_York",
):
    """
    Compare YTD call-type counts through `as_of` vs the same YTD window last year.
    - as_of: str|pd.Timestamp|None  (default=now in `tz`)
    - Returns a DataFrame with ThisPeriod, PrevPeriod, Change, % Change
    """

    # --- normalize timestamp inputs ---
    if as_of is None:
        as_of = pd.Timestamp.now(tz)
    else:
        as_of = pd.Timestamp(as_of)
        if as_of.tzinfo is None:
            as_of = as_of.tz_localize(tz)
        else:
            as_of = as_of.tz_convert(tz)

    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors="coerce")

    # bring data into the same tz (or localize if naive)
    if tz is not None:
        if df[datetime_col].dt.tz is None:
            df[datetime_col] = df[datetime_col].dt.tz_localize(tz)
        else:
            df[datetime_col] = df[datetime_col].dt.tz_convert(tz)

    # --- window for THIS period (YTD through as_of) ---
    this_start = pd.Timestamp(as_of.year, 1, 1, tz=as_of.tz)
    this_end = as_of

    # --- window for PREV period (YTD through same month/day last year) ---
    prev_year = as_of.year - 1
    prev_start = pd.Timestamp(prev_year, 1, 1, tz=as_of.tz)
    # handle Feb 29: clamp day to last day of month in prev_year
    last_day_prev_month = pd.Period(f"{prev_year}-{as_of.month:02d}").days_in_month
    prev_day_clamped = min(as_of.day, last_day_prev_month)
    prev_end = pd.Timestamp(prev_year, as_of.month, prev_day_clamped, 23, 59, 59, tz=as_of.tz)

    # --- filter and count ---
    this_counts = (
        df[(df[datetime_col] >= this_start) & (df[datetime_col] <= this_end)]
        .groupby(calltype_col, dropna=False)
        .size()
        .rename("ThisPeriod")
    )
    prev_counts = (
        df[(df[datetime_col] >= prev_start) & (df[datetime_col] <= prev_end)]
        .groupby(calltype_col, dropna=False)
        .size()
        .rename("PrevPeriod")
    )

    # pick categories using the union of top_n from both sides
    top_idx = (
        this_counts.nlargest(top_n).index
        .union(prev_counts.nlargest(top_n).index)
    )

    comp = pd.concat([this_counts, prev_counts], axis=1).reindex(top_idx).fillna(0).astype(int)
    comp["Change"] = comp["ThisPeriod"] - comp["PrevPeriod"]
    comp["% Change"] = np.where(
        comp["PrevPeriod"] > 0,
        (comp["Change"] / comp["PrevPeriod"] * 100).round(1),
        np.nan,  # or 100.0 if you prefer to show growth from zero
    )

    return comp.sort_values(by="ThisPeriod", ascending=False)

tbl = most_common_call_types_asof(df, as_of="2025-06-30", top_n=15)
print(tbl)


                     ThisPeriod  PrevPeriod  Change  % Change
CallType                                                     
Welfare Check              2160        2538    -378     -14.9
911 Hangup                 1730        2457    -727     -29.6
Disturbance                1287        1862    -575     -30.9
Parking Complaint           958        1077    -119     -11.0
Domestic                    946        1337    -391     -29.2
Alarm - Business            816         908     -92     -10.1
Accident                    668         754     -86     -11.4
Theft                       596         754    -158     -21.0
Suspicious Person           568         960    -392     -40.8
Unwanted Person             503         734    -231     -31.5
Assist Other Agency         448         739    -291     -39.4
Noise Complaint             430         570    -140     -24.6
Accident - Hit Skip         412         411       1       0.2
Alarm - Residential         361         488    -127     -26.0
Disabled

What times of day see the highest volume of calls by shift?

In [10]:
def call_volume_by_hour_and_shift(df):
    df = df.copy()
    df['LocalDatetime'] = pd.to_datetime(df['LocalDatetime'], errors='coerce')
    df['Hour'] = df['LocalDatetime'].dt.hour

    # Ensure Shift is present or assign it
    if 'Shift' not in df.columns:
        df['Shift'] = df['LocalDatetime'].apply(fn.assign_shift)

    # Group by Shift and Hour
    grouped = df.groupby(['Shift', 'Hour']).size().reset_index(name='CallVolume')

    # Pivot for better readability
    pivot_table = grouped.pivot(index='Hour', columns='Shift', values='CallVolume').fillna(0).astype(int)

    return pivot_table
pivot = call_volume_by_hour_and_shift(df.loc[df['LocalDatetime'].dt.year == 2025])
print(pivot)

Shift  1st Shift  2nd Shift  3rd Shift
Hour                                  
0            754          0          0
1            563          0          0
2            496          0          0
3            445          0          0
4            357          0          0
5            295          0          0
6              0        354          0
7              0        531          0
8              0        698          0
9              0        851          0
10             0        972          0
11             0       1024          0
12             0        980          0
13             0       1084          0
14             0          0       1095
15             0          0       1144
16             0          0       1167
17             0          0       1065
18             0          0        989
19             0          0        975
20             0          0        931
21             0          0        903
22           806          0          0
23           831         

Which addresses have the highest number of repeated calls?

In [11]:
def top_repeat_call_addresses(df, top_n=20):
    df = df.copy()
    df['FullAddress'] = df['FullAddress'].fillna('Unknown')

    # Count occurrences of each address
    address_counts = df['FullAddress'].value_counts().reset_index()
    address_counts.columns = ['FullAddress', 'CallCount']

    # Return top N
    return address_counts.head(top_n)
repeat_calls = top_repeat_call_addresses(df)
print(repeat_calls)


                         FullAddress  CallCount
0                         Not Listed       5856
1               4004 TUSCARAWAS ST W       2851
2                       3200 US 62         2846
3                     2600 6TH ST SW       1192
4                700 MCKINLEY AVE NW       1057
5   100 SOMEWHERE IN THE CITY   &           943
6                      221 3RD ST SW        932
7                    200 HIGH AVE SW        890
8                    1212 12TH ST NW        854
9                800 TUSCARAWAS ST W        809
10                     131 5TH ST NE        754
11                1114 GONDER AVE SE        678
12                   2421 13TH ST NW        604
13              2210 TUSCARAWAS ST W        602
14              3131 TUSCARAWAS ST W        591
15              2215 TUSCARAWAS ST E        589
16                  1320 MERCY DR NW        583
17              1700 GATEWAY BLVD SE        571
18                    205 19TH ST NE        554
19                     715 2ND ST NE    

What percentage of calls resulted in a report being taken or an arrest?

In [12]:
def percentage_report_or_arrest(df):
    df = df.copy()
    df['dispo'] = df['dispo'].fillna('').str.upper()

    total_calls = len(df)
    report_calls = df['dispo'].str.contains('REPORT TAKEN').sum()
    arrest_calls = df['dispo'].str.contains('ARREST').sum()

    # Avoid double-counting calls that include both
    report_or_arrest_calls = df[
        df['dispo'].str.contains('REPORT TAKEN') | df['dispo'].str.contains('ARREST')
    ]

    percent_report = (report_calls / total_calls) * 100
    percent_arrest = (arrest_calls / total_calls) * 100
    percent_combined = (len(report_or_arrest_calls) / total_calls) * 100

    return {
        "Total Calls": total_calls,
        "Report Taken %": round(percent_report, 2),
        "Arrest %": round(percent_arrest, 2),
        "Report or Arrest %": round(percent_combined, 2)
    }
results = percentage_report_or_arrest(df)
print(results)


{'Total Calls': 274855, 'Report Taken %': 24.56, 'Arrest %': 4.35, 'Report or Arrest %': 26.12}


In [13]:
def summarize_section(api_key, section_title, df_or_text):
    from langchain_openai import ChatOpenAI
    from langchain.chains import ConversationChain
    from langchain.prompts import PromptTemplate
    from langchain.memory import ConversationBufferMemory

    if isinstance(df_or_text, pd.DataFrame):
        data_str = df_or_text.to_string(index=False)
    elif isinstance(df_or_text, dict):
        data_str = "\n".join(f"{k}: {v}" for k, v in df_or_text.items())
    else:
        data_str = str(df_or_text)

    prompt_template = PromptTemplate(
        input_variables=["history", "input"],
        template=f"""You are a police analyst assistant. Provide a concise summary of the following section titled "{section_title}".
Focus on insights, trends, or red flags for command staff. Keep your summary under 100 words.

Conversation history:
{{history}}

Human: {{input}}
AI:"""
    )

    chain = ConversationChain(
        llm=ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.3),
        memory=ConversationBufferMemory(),
        prompt=prompt_template
    )

    input_text = f"Section Title: {section_title}\nData:\n{data_str}"
    return chain.run(input_text)

api_key = os.environ['OPEN_API_KEY']

summaries = []
#summaries.append(summarize_section(api_key, "Call Types This Year vs. Last Year", most_common_call_types_this_vs_last_year(df)))
summaries.append(summarize_section(api_key, "Hourly Call Volume by First Shift", call_volume_by_hour_and_shift(df)["1st Shift"].loc[lambda s: s > 0]))
summaries.append(summarize_section(api_key, "Hourly Call Volume by Second Shift", call_volume_by_hour_and_shift(df)["2nd Shift"].loc[lambda s: s > 0]))
summaries.append(summarize_section(api_key, "Hourly Call Volume by Third Shift", call_volume_by_hour_and_shift(df)["3rd Shift"].loc[lambda s: s > 0]))
summaries.append(summarize_section(api_key, "Top Repeat Call Addresses", top_repeat_call_addresses(df)))
summaries.append(summarize_section(api_key, "Report/Arrest Percentage. Not all calls need reports. Not all reports have arrests.", percentage_report_or_arrest(df)))

final_report = "\n\n".join(summaries)
print(final_report)


The data shows a significant increase in call volume during the late hours of the first shift (22 and 23). This may indicate a need for additional resources or staffing during those hours to handle the higher volume of calls. Command staff should consider adjusting staffing levels to address this trend and ensure timely response to calls during peak hours.

The hourly call volume for the second shift shows a steady increase from 6 PM to 1 PM, with a peak at 1 PM. Command staff should be aware of the high call volume during these hours and ensure appropriate staffing levels to handle the workload effectively. Additionally, monitoring the trends in call volume can help identify any patterns or issues that may require attention.

The hourly call volume for the third shift shows a consistent decrease from 14:00 to 21:00, with the highest volume at 16:00. Command staff should be aware of potential staffing adjustments needed during the later hours of the shift to ensure adequate response to

### Break out specifics

In [14]:
# Loads the daily summary of calltypes to ORI and Date
df_summary = fn.load_summary_data()

# Add call types to the Summary Data to focus on Reportable calls only
cfs_types = pd.read_excel("resources/cfs/lib_call_types.xlsx", usecols=["CallType","Reportable","CodeType"])
df_summary = df_summary.merge(cfs_types, on="CallType", how="left")
df_summary_reportable = df_summary.loc[df_summary['Reportable'] == True].copy()
df_summary_reportable['Year'] = pd.to_datetime(df_summary_reportable['Date']).dt.year
df_summary_reportable['DOY'] = pd.to_datetime(df_summary_reportable['Date']).dt.dayofyear

# analyze the data and cluster by call type
traffic_stops = df_summary.loc[df_summary['CallType'] == 'Traffic Stop'].copy()
shots_fired = df_summary.loc[df_summary['CallType'] == 'Shots Fired'].copy()
accidents = df_summary.loc[df_summary['CallType'].str.contains('Accident')].copy()

past_10_weeks = fn.compute_total_cfs_past_10_weeks(df_summary_reportable)

ytd_cfs = fn.compute_total_cfs_ytd(df_summary_reportable)
ytd_traffic_stops = fn.compute_total_cfs_ytd(traffic_stops)
ytd_shots_fired = fn.compute_total_cfs_ytd(shots_fired)
ytd_accidents = fn.compute_total_cfs_ytd(accidents)

In [15]:
df_summary_reportable

,ORI,Date,Shift,CallType,Total_Calls,Reportable,CodeType,Year,DOY,Month-Day
0,OH0760400,2020-01-01,1st Shift,911 Hangup,7,True,NaN,2020,1,01-01
1,OH0760400,2020-01-01,1st Shift,ATV Complaint,1,True,Nuisance,2020,1,01-01
2,OH0760400,2020-01-01,1st Shift,Accident - Hit Skip,1,True,Accident,2020,1,01-01
4,OH0760400,2020-01-01,1st Shift,Alarm - Business,2,True,NaN,2020,1,01-01
5,OH0760400,2020-01-01,1st Shift,Alarm - Residential,1,True,NaN,2020,1,01-01
...,...,...,...,...,...,...,...,...,...,...
176305,OH0760400,2025-06-08,3rd Shift,Welfare Check,10,True,NaN,2025,159,06-08
176306,OH0760400,2025-06-09,1st Shift,Drug/Alcohol Investigation,1,True,NaN,2025,160,06-09
176307,OH0760400,2025-06-09,1st Shift,Noise Complaint,1,True,Nuisance,2025,160,06-09
176309,OH0760400,2025-06-09,1st Shift,Suspicious Vehicle,1,True,NaN,2025,160,06-09


In [16]:
df_summary_reportable.loc[(df_summary_reportable['DOY'] <= 159) & (df_summary_reportable['CodeType'] == "Violent") & (df_summary_reportable['Year'] >=2024)].groupby(["CallType","Year"]).size().unstack(fill_value=0)

Year,2024,2025
CallType,,
Abuse/Neglect,23,35
Assault,277,260
Bomb Threat,2,3
Domestic,415,545
Fight,48,40
Homicide,2,1
Kidnapping/Abduction,11,14
Rape,26,31
Robbery,52,32


In [69]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


prompt_template = PromptTemplate(
    input_variables=["history", "input"],
    template=f"""Can you summarize the tabled data using percentages for change and output the summary to paragraph form? While some call types may have low counts, focus on the more significant changes over time.

Conversation history:
{{history}}

Human: {{input}}
AI:"""
)

chain = ConversationChain(
    llm=ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.3),
    memory=ConversationBufferMemory(),
    prompt=prompt_template
)

input_text = str(df_summary_reportable.loc[(df_summary_reportable['DOY'] <= 159) & (df_summary_reportable['CodeType'] == "Violent") & (df_summary_reportable['Year'] >=2024)].groupby(["CallType","Year"]).size().unstack(fill_value=0))
chain.run(input_text)

'Looking at the data, we can see that there have been some significant changes in call types from 2024 to 2025. The call types that saw the largest percentage increase in counts include Domestic (31%), Sex Offense (94%), and Threats (27%). On the other hand, some call types saw a decrease in counts, such as Robbery (38%) and Robbery - Armed (100%). Overall, it is clear that there has been a shift in the types of calls received by the authorities, with some call types becoming more prevalent while others are decreasing in frequency.'